In [1]:
#coding:utf-8

import pandas as pd
import time
import numpy as np
import warnings
import numpy as np
np.random.seed(2018)
warnings.filterwarnings("ignore")

# 时间处理
def time2cov(time_):
    '''
    时间是根据天数推移，所以日期为脱敏，但是时间本身不脱敏
    :param time_:
    :return:
    '''
    return time.strftime("%Y-%m-%d %H:%M:%S",time.localtime(time_))

print('train')
train = pd.read_csv('train.csv')
train = train.drop_duplicates(['instance_id'])
train = train.reset_index(drop=True)

print('test')
test_a = pd.read_csv('test.csv')

all_data = pd.concat([train,test_a])
all_data['real_time'] = pd.to_datetime(all_data['context_timestamp'].apply(time2cov))
all_data['real_hour'] = all_data['real_time'].dt.hour
all_data['real_day'] = all_data['real_time'].dt.day


train
test


In [2]:
def time_change(hour):
    hour = hour - 1
    if hour == -1:
        hour = 23
    return hour

def time_change_1(hour):
    hour = hour + 1
    if hour == 24:
        hour = 0
    return hour

all_data['hour_before'] = all_data['real_hour'].apply(time_change)
all_data['hour_after'] = all_data['real_hour'].apply(time_change_1)

# 18 21 19 20 22 23 24 | 25
print(all_data['real_day'].unique())

# train and test cov radio
# print(len((set(train['user_id']))&(set(test_a['user_id'])))/len(set(test_a['user_id'])))
# print(len((set(train['shop_id']))&(set(test_a['shop_id'])))/len(set(test_a['shop_id'])))
# print(len((set(train['item_id']))&(set(test_a['item_id'])))/len(set(test_a['item_id'])))
# user 0.26714801444043323
# shop 0.9781637717121588
# item 0.956427604871448

# shop feat

# item feat

# user feat


[18 21 19 20 22 23 24 25]


In [3]:
def c_log_loss(y_t,y_p):
    tmp = np.array(y_t) * np.log(np.array(y_p)) + (1 - np.array(y_t)) * np.log(1 - np.array(y_p))
    return -np.sum(tmp)/len(y_t),False

# 获取当前时间之前的前x天的转化率特征
def get_before_cov_radio(all_data,label_data,cov_list = list(['shop_id','item_id','real_hour','item_pv_level','item_sales_level']),day_list = list([1,2,3])):
    result = []
    r = pd.DataFrame()
    label_data_time = label_data['real_day'].min()
    label_data_time_set = label_data['real_day'].unique()
    print('label set day',label_data_time_set)
    for cov in cov_list:
        for d in day_list:
            feat_set = all_data[
                (all_data['real_day']>=label_data_time-d)&(all_data['real_day']<label_data_time)
                                ]
            print("cov feature",feat_set['real_day'].unique())
            print("cov time",cov)

            tmp = feat_set.groupby([cov],as_index=False).is_trade.agg({'mean':np.mean,'count':'count'}).add_suffix("_%s_before_%d_day"%(cov,d))

            tmp.rename(columns={'%s_%s_before_%d_day'%(cov,cov,d):cov},inplace=True)

            if d == 1:
                r = tmp
            else:
                r = pd.merge(r,tmp,on=[cov],how='outer').fillna(0)

        result.append(r)
    return result


def calc_categry_feat(data):
    data['item_category_list_1'] = data['item_category_list'].apply(lambda x: int(x.split(';')[0]))
    data['item_category_list_2'] = data['item_category_list'].apply(lambda x: int(x.split(';')[1]))
    data['item_property_list_0'] = data['item_property_list'].apply(lambda x: int(x.split(';')[0]))
    data['item_property_list_1'] = data['item_property_list'].apply(lambda x: int(x.split(';')[1]))
    data['item_property_list_2'] = data['item_property_list'].apply(lambda x: int(x.split(';')[2]))

    for i in range(3):
        data['predict_category_%d' % (i)] = data['predict_category_property'].apply(
            lambda x: int(str(x.split(";")[i]).split(":")[0]) if len(x.split(";")) > i else -1
        )

    for item_cate in ['item_category_list_1','item_category_list_2']:
        for pre_item_cate in ['predict_category_0','predict_category_1','predict_category_2']:
            data['%s_%s'%(item_cate,pre_item_cate)] = data[item_cate] == data[pre_item_cate]
            data['%s_%s'%(item_cate,pre_item_cate)] = data['%s_%s'%(item_cate,pre_item_cate)].astype(int)

    del data['item_category_list']
    del data['item_property_list']
    del data['predict_category_property']
    return data

take_columns = ['instance_id','item_id','shop_id','user_id','is_trade']

shop_current_col = [
    'shop_score_description','shop_score_delivery','shop_score_service',
    'shop_star_level','shop_review_positive_rate','shop_review_num_level'
]

user_col = [
    'user_gender_id','user_age_level','user_occupation_id','user_star_level'
]

item_col = [
    'item_brand_id','item_city_id','item_price_level',
    'item_sales_level','item_collected_level','item_pv_level',
    'item_category_list','item_property_list'
]
time_feat = ['real_hour','hour_before','hour_after','context_timestamp','real_day']

context_col = ['predict_category_property','context_page_id']

feat = take_columns + shop_current_col + time_feat + user_col + item_col + context_col



In [4]:
def get_history_user_feat(all_data,data):
    label_data_time = data['real_day'].min()
    print(label_data_time)

    tmp = all_data[all_data['real_day'] < label_data_time]
    print(tmp['real_day'].unique())

    user_time = tmp.groupby(['user_id'],as_index=False).context_timestamp.agg({'day_begin':'min','day_end':'max'})
    user_time['alive'] = user_time['day_end'] - user_time['day_begin']
    user_time['s_alive'] = label_data_time - user_time['day_begin']
    user_time['alive/s_alive'] =  user_time['alive'] / user_time['s_alive']

    user_time_cov = tmp[tmp['is_trade']==1]
    user_time_cov = user_time_cov.groupby(['user_id'], as_index=False).context_timestamp.agg({'day_end_cov': 'max'})

    user_time_cov = pd.DataFrame(user_time_cov).drop_duplicates(['user_id','day_end_cov'])

    data = pd.merge(data,user_time[['user_id','alive','s_alive','alive/s_alive','day_begin','day_end']],on=['user_id'],how='left')

    data = pd.merge(data,user_time_cov,on=['user_id'],how='left')
    data['day_end_cov'] = data['day_end_cov'].fillna(data['day_end'])

    data['alive_cov'] = data['day_end_cov'] - data['day_begin']
    data['alive/alive_cov'] = data['alive'] / data['alive_cov']
    # data['s_alive/alive_cov'] = data['s_alive'] / data['alive_cov']

    del data['day_end_cov']
    del data['day_end']
    del data['day_begin']

    # for i in [1,2,3]:
    #     tmp = all_data[(all_data['real_day'] < data['real_day'].min()) & (all_data['real_day'] >= data['real_day'].min() - i)]
    #     user_item_sales_level_day = tmp.groupby(['user_id'], as_index=False)['item_sales_level'] \
    #         .agg({'user_item_sales_level_day_mean': 'mean',
    #               'user_item_sales_level_day_median': 'median',
    #               'user_item_sales_level_day_min': 'min',
    #               'user_item_sales_level_day_max': 'max',
    #               'user_item_sales_level_day_std': 'std',
    #               'user_item_sales_level_day_count': 'count'})
    #     data = pd.merge(data, user_item_sales_level_day, 'left', on=['user_id'])

    # data = data[['user_id','alive','s_alive','alive/s_alive','alive_cov','alive/alive_cov']]

    return data.fillna(-1)


def get_history_shop_feat(all_data,data):
    label_data_time = data['real_day'].min()
    print(label_data_time)
    for i in [1,2,3]:
        tmp = all_data[(all_data['real_day'] < label_data_time)&(all_data['real_day'] >= label_data_time - i)]

        shop_score_service_hour = tmp.groupby(['real_hour'], as_index=False)[
            'shop_score_service'] \
            .agg({
                  'shop_score_service_hour_std_%d'%(i): 'std',
               })
        data = pd.merge(data, shop_score_service_hour, 'left', on=['real_hour'])

        shop_score_delivery = tmp.groupby(['real_hour'], as_index=False)[
            'shop_score_delivery'] \
            .agg({
            'shop_score_delivery_hour_std_%d' % (i): 'std',
        })
        data = pd.merge(data, shop_score_delivery, 'left', on=['real_hour'])

        shop_score_service_hour = tmp.groupby(['real_hour'], as_index=False)[
            'shop_score_description'] \
            .agg({
            'shop_score_description_hour_std_%d' % (i): 'std',
        })
        data = pd.merge(data, shop_score_service_hour, 'left', on=['real_hour'])

        shop_review_positive_rate = tmp.groupby(['real_hour'], as_index=False)[
            'shop_review_positive_rate'] \
            .agg({
            'shop_review_positive_rate_hour_std_%d' % (i): 'std',
        })
        data = pd.merge(data, shop_review_positive_rate, 'left', on=['real_hour'])

        shop_star_level = tmp.groupby(['real_hour'], as_index=False)[
            'shop_star_level'] \
            .agg({
            'shop_star_level_hour_std_%d' % (i): 'std',
        })
        data = pd.merge(data, shop_star_level, 'left', on=['real_hour'])

        shop_review_num_level = tmp.groupby(['real_hour'], as_index=False)[
            'shop_review_num_level'] \
            .agg({
            'shop_review_num_level_hour_std_%d' % (i): 'std',
        })
        data = pd.merge(data, shop_review_num_level, 'left', on=['real_hour'])

        shop_query_day_hour = tmp.groupby(['shop_id', 'real_hour']).size().reset_index().rename(
            columns={0: 'shop_query_day_hour_%d'%(i)})
        data = pd.merge(data, shop_query_day_hour, 'left', on=['shop_id', 'real_hour'])

    return data


def get_history_item_feat(all_data,data):
    for i in [1, 2, 3]:
        tmp = all_data[(all_data['real_day']<data['real_day'].min())&(all_data['real_day']>=data['real_day'].min()-i)]

        item_brand_id_day = tmp.groupby(['item_city_id','real_hour']).size().reset_index().rename(
            columns={0: 'item_brand_id_day_%d'%(i)})
        data = pd.merge(data, item_brand_id_day, 'left', on=['item_city_id','real_hour'])

        item_brand_id_hour = tmp.groupby(['item_brand_id', 'real_hour']).size().reset_index().rename(
            columns={0: 'item_brand_id_hour_%d'%(i)})
        data = pd.merge(data, item_brand_id_hour, 'left', on=['item_brand_id', 'real_hour'])
        item_pv_level_hour = tmp.groupby(['item_pv_level', 'real_hour']).size().reset_index().rename(
            columns={0: 'item_pv_level_hour_%d'%(i)})
        data = pd.merge(data, item_pv_level_hour, 'left', on=['item_pv_level','real_hour'])
        #
        # item_pv_level_day = data.groupby(['real_day','real_hour'], as_index=False)['item_pv_level'] \
        #     .agg({'item_pv_level_day_mean_%d'%(i): 'mean',
        #           'item_pv_level_day_median_%d'%(i): 'median',
        #           'item_pv_level_day_std_%d'%(i): 'std'
        #           })
        # data = pd.merge(data, item_pv_level_day, 'left', on=['real_day','real_hour'])
    return data

print('make feat')
def make_feat(data,feat):
    '''
    :param data: 标签数据，当前时刻的用户特征
    :param feat: 特征数据，统计的用户特征
    :return: 拼接后的特征
    '''

    data = calc_categry_feat(data)
    data = get_history_user_feat(all_data,data)
    data = get_history_shop_feat(all_data,data)
    data = get_history_item_feat(all_data,data)

    for f in feat:
        data = pd.merge(data,f,on=[f.columns[0]],how='left')

    return data.fillna(0)


make feat


In [5]:

test_a = all_data[train.shape[0]:]

train = all_data[:train.shape[0]]
val_a = train[train['real_day']==24]
train_a = train[train['real_day']==23]
train_b = train[train['real_day']==22]
train_c = train[train['real_day']==21]

# 传入全部数据和当前标签数据
test_cov_feat = get_before_cov_radio(all_data,test_a)
val_cov_feat = get_before_cov_radio(all_data,val_a)

train_cov_feat_a = get_before_cov_radio(all_data,train_a)
train_cov_feat_b = get_before_cov_radio(all_data,train_b)
train_cov_feat_c = get_before_cov_radio(all_data,train_c)


train_a = make_feat(train_a[feat],train_cov_feat_a)
train_b = make_feat(train_b[feat],train_cov_feat_b)
train_c = make_feat(train_c[feat],train_cov_feat_c)

test_a = make_feat(test_a[feat],test_cov_feat)
val_a = make_feat(val_a[feat],val_cov_feat)


train = pd.concat([train_a,train_b])
train = pd.concat([train,train_c])

# print(train.shape)
# train = pd.concat([train,val_a])
# print(train.shape)

y_train = train.pop('is_trade')
train_index = train.pop('instance_id')
X_train = train

y_test = test_a.pop('is_trade')
test_index = test_a.pop('instance_id')
X_test = test_a

y_val = val_a.pop('is_trade')
val_index = val_a.pop('instance_id')
X_val = val_a


# print(train.head())

category_list = [
    'item_id','shop_id','user_id','user_gender_id','user_age_level',
     'user_occupation_id','user_star_level',
    'item_brand_id', 'item_city_id', 'item_price_level',
    'item_sales_level', 'item_collected_level', 'item_pv_level',
    'shop_review_num_level','shop_star_level','item_category_list_1','item_category_list_2',
    'item_property_list_0','item_property_list_1','item_property_list_2',
    'predict_category_0','predict_category_1','predict_category_2','context_page_id'
]

def make_cat(data):
    for i in category_list:
        data[i] = data[i].astype('category')
    return data


train_test_val = pd.concat([X_train,X_test])
train_test_val = pd.concat([train_test_val,X_val])
train_test_val = train_test_val.reset_index(drop=True)

# train_test_val = make_cat(train_test_val)
#
# X_train = train_test_val[:X_train.shape[0]]
# X_test = train_test_val[X_train.shape[0]:X_train.shape[0]+X_test.shape[0]]
# X_val = train_test_val[X_train.shape[0]+X_test.shape[0]:]

X_train = make_cat(X_train)
X_test = make_cat(X_test)
X_val = make_cat(X_val)

print(X_train.shape)
print(X_test.shape)
print(X_val.shape)


# X_test = make_cat(X_test)
# X_val = make_cat(X_val)

del X_train['hour_before']
del X_test['hour_before']
del X_val['hour_before']

del X_train['hour_after']
del X_test['hour_after']
del X_val['hour_after']

del X_train['real_day']
del X_test['real_day']
del X_val['real_day']

print(X_train.dtypes)

del X_train['context_timestamp']
del X_test['context_timestamp']
del X_val['context_timestamp']

X_train = X_train[X_train.columns]
X_test = X_test[X_train.columns]
X_val = X_val[X_train.columns]



label set day [25]
cov feature [24]
cov time shop_id
cov feature [23 24]
cov time shop_id
cov feature [22 23 24]
cov time shop_id
cov feature [24]
cov time item_id
cov feature [23 24]
cov time item_id
cov feature [22 23 24]
cov time item_id
cov feature [24]
cov time real_hour
cov feature [23 24]
cov time real_hour
cov feature [22 23 24]
cov time real_hour
cov feature [24]
cov time item_pv_level
cov feature [23 24]
cov time item_pv_level
cov feature [22 23 24]
cov time item_pv_level
cov feature [24]
cov time item_sales_level
cov feature [23 24]
cov time item_sales_level
cov feature [22 23 24]
cov time item_sales_level
label set day [24]
cov feature [23]
cov time shop_id
cov feature [22 23]
cov time shop_id
cov feature [21 22 23]
cov time shop_id
cov feature [23]
cov time item_id
cov feature [22 23]
cov time item_id
cov feature [21 22 23]
cov time item_id
cov feature [23]
cov time real_hour
cov feature [22 23]
cov time real_hour
cov feature [21 22 23]
cov time real_hour
cov feature [23]


In [6]:
import lightgbm as lgb
#
# 线下学习
gbm = lgb.LGBMRegressor(objective='binary',
                        num_leaves=32,
                        learning_rate=0.01,
                        n_estimators=2000,
                        colsample_bytree = 0.65,
                        subsample = 0.65,
                        seed=0
                        )
gbm.fit(X_train,y_train,
    eval_set=[(X_val, y_val)],
    eval_metric=['binary_logloss'],
    early_stopping_rounds= 200)


[1]	valid_0's binary_logloss: 0.683874
Training until validation scores don't improve for 200 rounds.
[2]	valid_0's binary_logloss: 0.674778
[3]	valid_0's binary_logloss: 0.66584
[4]	valid_0's binary_logloss: 0.657098
[5]	valid_0's binary_logloss: 0.648517
[6]	valid_0's binary_logloss: 0.640088
[7]	valid_0's binary_logloss: 0.631887
[8]	valid_0's binary_logloss: 0.623785
[9]	valid_0's binary_logloss: 0.615859
[10]	valid_0's binary_logloss: 0.608062
[11]	valid_0's binary_logloss: 0.600415
[12]	valid_0's binary_logloss: 0.592876
[13]	valid_0's binary_logloss: 0.585488
[14]	valid_0's binary_logloss: 0.578236
[15]	valid_0's binary_logloss: 0.571132
[16]	valid_0's binary_logloss: 0.56414
[17]	valid_0's binary_logloss: 0.557265
[18]	valid_0's binary_logloss: 0.55052
[19]	valid_0's binary_logloss: 0.543895
[20]	valid_0's binary_logloss: 0.537352
[21]	valid_0's binary_logloss: 0.530939
[22]	valid_0's binary_logloss: 0.524639
[23]	valid_0's binary_logloss: 0.518449
[24]	valid_0's binary_logloss

[204]	valid_0's binary_logloss: 0.12334
[205]	valid_0's binary_logloss: 0.122797
[206]	valid_0's binary_logloss: 0.122259
[207]	valid_0's binary_logloss: 0.121728
[208]	valid_0's binary_logloss: 0.121197
[209]	valid_0's binary_logloss: 0.120682
[210]	valid_0's binary_logloss: 0.120171
[211]	valid_0's binary_logloss: 0.119672
[212]	valid_0's binary_logloss: 0.119175
[213]	valid_0's binary_logloss: 0.118684
[214]	valid_0's binary_logloss: 0.1182
[215]	valid_0's binary_logloss: 0.117715
[216]	valid_0's binary_logloss: 0.117237
[217]	valid_0's binary_logloss: 0.11677
[218]	valid_0's binary_logloss: 0.116305
[219]	valid_0's binary_logloss: 0.11585
[220]	valid_0's binary_logloss: 0.115399
[221]	valid_0's binary_logloss: 0.114956
[222]	valid_0's binary_logloss: 0.114514
[223]	valid_0's binary_logloss: 0.114079
[224]	valid_0's binary_logloss: 0.113642
[225]	valid_0's binary_logloss: 0.113226
[226]	valid_0's binary_logloss: 0.11281
[227]	valid_0's binary_logloss: 0.112394
[228]	valid_0's binary

[403]	valid_0's binary_logloss: 0.0844247
[404]	valid_0's binary_logloss: 0.0843949
[405]	valid_0's binary_logloss: 0.0843667
[406]	valid_0's binary_logloss: 0.0843347
[407]	valid_0's binary_logloss: 0.084308
[408]	valid_0's binary_logloss: 0.0842811
[409]	valid_0's binary_logloss: 0.0842557
[410]	valid_0's binary_logloss: 0.0842221
[411]	valid_0's binary_logloss: 0.0841934
[412]	valid_0's binary_logloss: 0.0841683
[413]	valid_0's binary_logloss: 0.0841448
[414]	valid_0's binary_logloss: 0.0841171
[415]	valid_0's binary_logloss: 0.0840925
[416]	valid_0's binary_logloss: 0.084064
[417]	valid_0's binary_logloss: 0.0840355
[418]	valid_0's binary_logloss: 0.0840077
[419]	valid_0's binary_logloss: 0.0839846
[420]	valid_0's binary_logloss: 0.0839646
[421]	valid_0's binary_logloss: 0.0839411
[422]	valid_0's binary_logloss: 0.0839146
[423]	valid_0's binary_logloss: 0.0838891
[424]	valid_0's binary_logloss: 0.0838658
[425]	valid_0's binary_logloss: 0.0838447
[426]	valid_0's binary_logloss: 0.08

[598]	valid_0's binary_logloss: 0.083342
[599]	valid_0's binary_logloss: 0.0833457
[600]	valid_0's binary_logloss: 0.0833507
[601]	valid_0's binary_logloss: 0.0833517
[602]	valid_0's binary_logloss: 0.0833543
[603]	valid_0's binary_logloss: 0.0833539
[604]	valid_0's binary_logloss: 0.0833502
[605]	valid_0's binary_logloss: 0.0833515
[606]	valid_0's binary_logloss: 0.0833542
[607]	valid_0's binary_logloss: 0.0833587
[608]	valid_0's binary_logloss: 0.0833628
[609]	valid_0's binary_logloss: 0.0833691
[610]	valid_0's binary_logloss: 0.0833729
[611]	valid_0's binary_logloss: 0.0833755
[612]	valid_0's binary_logloss: 0.0833776
[613]	valid_0's binary_logloss: 0.0833845
[614]	valid_0's binary_logloss: 0.0833911
[615]	valid_0's binary_logloss: 0.0833951
[616]	valid_0's binary_logloss: 0.0834009
[617]	valid_0's binary_logloss: 0.083407
[618]	valid_0's binary_logloss: 0.0834104
[619]	valid_0's binary_logloss: 0.0834133
[620]	valid_0's binary_logloss: 0.0834214
[621]	valid_0's binary_logloss: 0.08

LGBMRegressor(boosting_type='gbdt', class_weight=None, colsample_bytree=0.65,
       learning_rate=0.01, max_depth=-1, min_child_samples=20,
       min_child_weight=0.001, min_split_gain=0.0, n_estimators=2000,
       n_jobs=-1, num_leaves=32, objective='binary', random_state=None,
       reg_alpha=0.0, reg_lambda=0.0, seed=0, silent=True, subsample=0.65,
       subsample_for_bin=200000, subsample_freq=1)

In [8]:

imp = pd.DataFrame()
imp['n'] = list(X_train.columns)
imp['s'] = list(gbm.feature_importances_)
print(imp.sort_values('s',ascending=False))

print('Start predicting...')
# predict
y_pred_1 = gbm.predict(X_val, num_iteration=gbm.best_iteration_)
y_tt = gbm.predict(X_train, num_iteration=gbm.best_iteration_)


                                          n     s
0                                   item_id  2737
1                                   shop_id  1425
14                            item_brand_id   599
74                mean_shop_id_before_3_day   536
80                mean_item_id_before_3_day   392
82              mean_real_hour_before_1_day   365
86              mean_real_hour_before_3_day   364
78                mean_item_id_before_2_day   336
72                mean_shop_id_before_2_day   323
84              mean_real_hour_before_2_day   302
77               count_item_id_before_1_day   240
94       mean_item_sales_level_before_1_day   233
4                       shop_score_delivery   213
96       mean_item_sales_level_before_2_day   207
47            shop_score_service_hour_std_2   205
79               count_item_id_before_2_day   197
43     shop_review_positive_rate_hour_std_1   194
54            shop_score_service_hour_std_3   192
57     shop_review_positive_rate_hour_std_3   191


In [9]:

from sklearn.metrics import log_loss

print(log_loss(y_val,y_pred_1))

print(log_loss(y_train,y_tt))


0.083132297817
0.0781307660134


In [12]:

# 线上提交

gbm_sub = lgb.LGBMRegressor(objective='binary',
                        num_leaves=32,
                        learning_rate=0.01,
                        n_estimators = gbm.best_iteration_+1,
                        colsample_bytree = 0.65,
                        subsample = 0.65,
                        seed=0
                        )

X_train = pd.concat([X_train,X_val])
y_train = pd.concat([y_train,y_val])

X_train = make_cat(X_train)

X_train = X_train[X_train.columns]

gbm_sub.fit(X_train,y_train,
    eval_set=[(X_train, y_train)],
    eval_metric=['binary_logloss'])

y_sub_1 = gbm_sub.predict(X_test)

y_tt = gbm_sub.predict(X_train, num_iteration=gbm_sub.best_iteration_)


[1]	training's binary_logloss: 0.683892
[2]	training's binary_logloss: 0.67482
[3]	training's binary_logloss: 0.665925
[4]	training's binary_logloss: 0.657198
[5]	training's binary_logloss: 0.648645
[6]	training's binary_logloss: 0.640251
[7]	training's binary_logloss: 0.632011
[8]	training's binary_logloss: 0.623929
[9]	training's binary_logloss: 0.615997
[10]	training's binary_logloss: 0.608214
[11]	training's binary_logloss: 0.600575
[12]	training's binary_logloss: 0.593073
[13]	training's binary_logloss: 0.585711
[14]	training's binary_logloss: 0.578479
[15]	training's binary_logloss: 0.571372
[16]	training's binary_logloss: 0.564394
[17]	training's binary_logloss: 0.557532
[18]	training's binary_logloss: 0.550798
[19]	training's binary_logloss: 0.544176
[20]	training's binary_logloss: 0.537674
[21]	training's binary_logloss: 0.531276
[22]	training's binary_logloss: 0.524991
[23]	training's binary_logloss: 0.51881
[24]	training's binary_logloss: 0.512736
[25]	training's binary_logl

[201]	training's binary_logloss: 0.125915
[202]	training's binary_logloss: 0.125342
[203]	training's binary_logloss: 0.124773
[204]	training's binary_logloss: 0.124213
[205]	training's binary_logloss: 0.123666
[206]	training's binary_logloss: 0.123119
[207]	training's binary_logloss: 0.122571
[208]	training's binary_logloss: 0.122035
[209]	training's binary_logloss: 0.12151
[210]	training's binary_logloss: 0.120988
[211]	training's binary_logloss: 0.12047
[212]	training's binary_logloss: 0.119957
[213]	training's binary_logloss: 0.119455
[214]	training's binary_logloss: 0.118953
[215]	training's binary_logloss: 0.118463
[216]	training's binary_logloss: 0.117975
[217]	training's binary_logloss: 0.117492
[218]	training's binary_logloss: 0.117018
[219]	training's binary_logloss: 0.116545
[220]	training's binary_logloss: 0.116095
[221]	training's binary_logloss: 0.115634
[222]	training's binary_logloss: 0.115179
[223]	training's binary_logloss: 0.114737
[224]	training's binary_logloss: 0.1

[395]	training's binary_logloss: 0.0813944
[396]	training's binary_logloss: 0.0813302
[397]	training's binary_logloss: 0.081263
[398]	training's binary_logloss: 0.081195
[399]	training's binary_logloss: 0.0811341
[400]	training's binary_logloss: 0.0810654
[401]	training's binary_logloss: 0.080996
[402]	training's binary_logloss: 0.0809301
[403]	training's binary_logloss: 0.0808638
[404]	training's binary_logloss: 0.0808035
[405]	training's binary_logloss: 0.0807395
[406]	training's binary_logloss: 0.0806863
[407]	training's binary_logloss: 0.0806267
[408]	training's binary_logloss: 0.0805645
[409]	training's binary_logloss: 0.0805033
[410]	training's binary_logloss: 0.080436
[411]	training's binary_logloss: 0.0803769
[412]	training's binary_logloss: 0.0803122
[413]	training's binary_logloss: 0.0802433
[414]	training's binary_logloss: 0.0801805
[415]	training's binary_logloss: 0.0801245
[416]	training's binary_logloss: 0.0800678
[417]	training's binary_logloss: 0.0800136
[418]	training'

TypeError: '<=' not supported between instances of 'NoneType' and 'int'

In [ ]:

# from sklearn.metrics import log_loss
# print(log_loss(y_train,y_tt))

# sub = pd.DataFrame()
# sub['instance_id'] = list(test_index)

# sub['predicted_score'] = list(y_sub_1)
 
# sub.to_csv('20180409.txt',sep=" ",index=False)